In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import gkey
gmaps.configure(api_key = gkey)

In [2]:
city_df = pd.read_csv("city.csv")
print(f"number of records: {len(city_df)}")
city_df.dropna(how = "any", inplace = True)
print(f"number of records after removal of nulls: {len(city_df)}")

number of records: 874
number of records after removal of nulls: 868


In [3]:
locations = city_df[["Latitude", "Longitude"]]
weights = city_df["Humidity"]

In [4]:
fig = gmaps.figure(center = (0,0), zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights = weights, dissipating = False, max_intensity = 200, point_radius = 5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
hotel_df = city_df.loc[(city_df["Max Temperature"] >= 23) & (city_df["Max Temperature"] < 33) & (city_df["Humidity"] < 35) & (city_df["Wind Speed"] < 15), :]
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Lat"] = ""
hotel_df["Hotel Lng"] = ""
hotel_df

<ipython-input-5-90ee5aa89e22>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""
<ipython-input-5-90ee5aa89e22>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Lat"] = ""
<ipython-input-5-90ee5aa89e22>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

,City,Max Temperature,Humidity,Wind Speed,Cloudiness,Latitude,Longitude,Country,Date,Hotel Name,Hotel Lat,Hotel Lng
105,kuytun,24.27,20.0,14.92,3.0,-22.4667,-68.9333,CL,1643406210,,,
121,dikson,25.39,22.0,2.99,99.0,7.8500,9.7833,NG,1643406323,,,
208,cabo san lucas,29.20,33.0,3.45,91.0,1.7471,40.0573,KE,1643406474,,,
286,kedrovyy,23.08,12.0,1.16,100.0,20.5022,-10.0711,MR,1643406444,,,
300,meadow lake,31.21,32.0,4.92,8.0,-31.5375,-68.5364,AR,1643406499,,,
324,mokhsogollokh,23.72,20.0,1.54,0.0,34.6391,-120.4579,US,1643406695,,,
376,porto walter,23.37,18.0,4.96,8.0,10.4795,16.7117,TD,1643406772,,,
424,kamenka,29.63,30.0,2.81,0.0,-40.9167,-73.1667,CL,1643406847,,,
439,carballo,25.59,24.0,3.09,0.0,-34.4260,-58.5796,AR,1643406852,,,
461,taltal,24.31,12.0,2.24,100.0,33.7000,-117.7675,US,1643406919,,,


In [6]:
radius = 5000
target_search = "hotel"

In [7]:
# params dictionary to update each iteration
params = {
    "radius": radius,
    "types": "lodging",
    "keyword": target_search,
    "key": gkey
}

# Use the lat/lng pulled in for loop
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel", using our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and convert into json, using pprint to enable data structure
    hotel_address = requests.get(base_url, params=params)
    hotel_address = hotel_address.json()
    #pprint(hotel_address)
    
    # leverage try/except in case data element empty. Use pass to skip exception.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Lat"] = hotel_address["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Hotel Lng"] = hotel_address["results"][0]["geometry"]["location"]["lng"]
        # print updated df with hotel name to validate df updated properly
        #print(hotel_df["Hotel Name"], hotel_address["results"][0]["name"])
    except (KeyError, IndexError):
        pass
    
print(hotel_df)

C:\Users\freed\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


               City  Max Temperature  Humidity  Wind Speed  Cloudiness  \
105          kuytun            24.27      20.0       14.92         3.0   
121          dikson            25.39      22.0        2.99        99.0   
208  cabo san lucas            29.20      33.0        3.45        91.0   
286        kedrovyy            23.08      12.0        1.16       100.0   
300     meadow lake            31.21      32.0        4.92         8.0   
324   mokhsogollokh            23.72      20.0        1.54         0.0   
376    porto walter            23.37      18.0        4.96         8.0   
424         kamenka            29.63      30.0        2.81         0.0   
439        carballo            25.59      24.0        3.09         0.0   
461          taltal            24.31      12.0        2.24       100.0   
467   novaya ladoga            23.49      21.0        4.36       100.0   
479     sungaipenuh            28.03      23.0        1.09         4.0   
484    todos santos            30.98  

In [8]:
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "", :]
print(f"Number of Desirable Hotels: {len(hotel_df)}")

marker_locations = hotel_df[["Hotel Lat", "Hotel Lng"]]
marker_locations

hotel_list = hotel_df["Hotel Name"]
hotel_nm_df = pd.DataFrame(hotel_list)
hotel_nm_df.dropna(how = "any", inplace = True)
hotel_list = hotel_nm_df["Hotel Name"].tolist()

Number of Desirable Hotels: 22


In [9]:
fig = gmaps.figure(center = (0,0), zoom_level = 2)
heat_layer = gmaps.heatmap_layer(locations, weights = weights, dissipating = False, max_intensity = 200, point_radius = 5)
markers = gmaps.marker_layer(marker_locations, info_box_content = [f"{hotel_name}" for hotel_name in hotel_list])
fig.add_layer(markers)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """
# # Store the DataFrame Row
# # NOTE: be sure to update with your DataFrame name
# hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# locations = hotel_df[["Lat", "Lng"]]